# Anagrams

In this exercise we reiterate over the previous Anagrams example from TP1.
We run a local Spark program to solve the exercise.

Let's start by importing the Spark Context:

In [ ]:
from pyspark import SparkContext

Next, we create an instance of SparkContext:

In [ ]:
sc = SparkContext()

Now we want to get our data from HDFS.
Let's check whether the file is already there:

> Note that by starting the line with `!` we run a bash command instead of python code.

In [ ]:
!hadoop fs -ls common_words_en_subset.txt

If the file don't show up you can put it on HDFS:

In [ ]:
!hadoop fs -put /vagrant/tp/1/common_words_en_subset.txt

Next, we load the data into a Spark RDD (note that this command is lazily evaluated)

In [ ]:
words = sc.textFile('common_words_en_subset.txt')
words

Let's display the content of the `words` RDD:

In [ ]:
words.collect()

Observe the results; make sure the data has been successfully loaded.
Then, run the map transformation:

In [ ]:
tuples = words.map(lambda x: (''.join(sorted(list(x))), x))
tuples.collect()

Then, run the groupByKey (the rough equivalent to Hadoop’s « shuffle »):

In [ ]:
grouped = tuples.groupByKey().mapValues(lambda x: list(x))
grouped.collect()

Observe the results. Then, run filter:

In [ ]:
filtered = grouped.filter(lambda x: len(x[1])>1)
filtered.collect()

Then, let's run mapValues:

In [ ]:
res1 = filtered.mapValues(lambda x: ", ".join(x))
res1.collect()

Finally, also apply mapValues to the unfiltered RDD and save both datasets on
disk:

In [ ]:
res2 = grouped.mapValues(lambda x: ", ".join(x))
res2.collect()

In [ ]:
res1.saveAsTextFile('res-words-filtered')
res2.saveAsTextFile('res-words-unfiltered')

Let's check that the results were written on HDFS:

In [ ]:
!hadoop fs -ls res-words-filtered

In [ ]:
!hadoop fs -ls res-words-unfiltered

In [ ]:
!hadoop fs -cat res-words-filtered/*

In [ ]:
!hadoop fs -cat res-words-unfiltered/*

Question: Why is the `grouped` RDD executed twice? How to force Spark to evaluate the `grouped` RDD only once?

You can also of course try out various other operations described during the course to familiarize yourself with them.